# Train

In [ ]:
import pandas as pd
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)
from tensorflow.keras import layers

In [ ]:
gambling = pd.read_csv("output/0.csv")
gambling.head()
gambling_features = gambling.copy()
gambling_labels = gambling_features.pop('Did_YOU_Win')

In [ ]:
inputs = {}

for name, column in gambling_features.items():
  dtype = column.dtype
  if dtype == object:
    dtype = tf.string
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

inputs

In [ ]:
numeric_inputs = {name:input for name,input in inputs.items()
                  if input.dtype==tf.float32}

x = layers.Concatenate()(list(numeric_inputs.values()))
norm = layers.Normalization()
norm.adapt(np.array(gambling[numeric_inputs.keys()]))
all_numeric_inputs = norm(x)

all_numeric_inputs

In [ ]:
preprocessed_inputs = [all_numeric_inputs]

In [ ]:
for name, input in inputs.items():
  if input.dtype == tf.float32:
    continue

  lookup = layers.StringLookup(vocabulary=np.unique(gambling_features[name]))
  one_hot = layers.CategoryEncoding(num_tokens=lookup.vocabulary_size())

  x = lookup(input)
  x = one_hot(x)
  preprocessed_inputs.append(x)

In [ ]:
preprocessed_inputs_cat = layers.Concatenate()(preprocessed_inputs)

gambling_preprocessing = tf.keras.Model(inputs, preprocessed_inputs_cat)

tf.keras.utils.plot_model(model = gambling_preprocessing , rankdir="LR", dpi=72, show_shapes=True)

In [ ]:
gambling_features_dict = {name: np.array(value) 
                         for name, value in gambling_features.items()}

In [ ]:
features_dict = {name:values[:1] for name, values in gambling_features_dict.items()}
gambling_preprocessing(features_dict)

In [ ]:
def gambling_model(preprocessing_head, inputs):
  body = tf.keras.Sequential([
    layers.Dense(64),
    layers.Dense(1),
  ])

  preprocessed_inputs = preprocessing_head(inputs)
  result = body(preprocessed_inputs)
  model = tf.keras.Model(inputs, result)

  model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam())
  return model

gambling_model = gambling_model(gambling_preprocessing, inputs)

In [ ]:
gambling_model.fit(x=gambling_features_dict, y=gambling_labels, epochs=1000)

In [ ]:
gambling_model.save('gambling.keras')

# Predictions

In [ ]:
import pandas as pd

# Load the gambling DataFrame from the CSV file
gambling = pd.read_csv("0.csv")

# Create the gambling_features variable
gambling_features = gambling.copy()
gambling_labels = gambling_features.pop('Did_YOU_Win')


In [ ]:
import tensorflow as tf
reloaded = tf.keras.models.load_model('gambling.keras')

data = [['2 of Hearts', '3 of Clubs', '3 of Spades', 'Jack of Clubs', 'Queen of Spades', '2 of Clubs', '7 of Clubs', 'King of Clubs', '5 of Diamonds', '5 of Clubs', '4 of Hearts', '4 of Diamonds', 'Ace of Clubs', 'Queen of Hearts', 'Jack of Diamonds', '10 of Hearts'],
        "2 of Hearts",
        0,
        2]
# print(gambling_features)
# Create a dictionary with the feature names as keys and the input data as values
input_dict = {name: tf.convert_to_tensor([value]) for name, value in zip(gambling_features.columns, data)}

# Make a prediction using the input dictionary
predictions = gambling_model.predict(input_dict)

# Apply the sigmoid function to obtain a probability
probabilities = tf.math.sigmoid(predictions)

# Extract the probability value
probability = probabilities.numpy()[0][0]

print(probability)


# Misc

In [ ]:
import tensorflow as tf

data = [['2 of Hearts', '3 of Clubs', '3 of Spades', 'Jack of Clubs', 'Queen of Spades', '2 of Clubs', '7 of Clubs', 'King of Clubs', '5 of Diamonds', '5 of Clubs', '4 of Hearts', '4 of Diamonds', 'Ace of Clubs', 'Queen of Hearts', 'Jack of Diamonds', '10 of Hearts'],
        "2 of Hearts",
        0,
        2]

# Create a dictionary with the feature names as keys and the input data as values
input_dict = {name: tf.convert_to_tensor([value]) for name, value in zip(gambling_features.columns, data)}

# Make a prediction using the input dictionary
predictions = gambling_model.predict(input_dict)

print(predictions)

In [ ]:
!pip install tensorflow_decision_forests
import tensorflow_decision_forests as tfdf

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math

In [ ]:
reloaded = tf.keras.models.load_model('gambling.keras')

pd_serving_dataset = pd.DataFrame({
    'Drawn_Cards': [str(['2 of Hearts', '3 of Clubs', '3 of Spades', 'Jack of Clubs', 'Queen of Spades', '2 of Clubs', '7 of Clubs', 'King of Clubs', '5 of Diamonds', '5 of Clubs', '4 of Hearts', '4 of Diamonds', 'Ace of Clubs', 'Queen of Hearts', 'Jack of Diamonds', '10 of Hearts'])],
    'Dealer_Up': ["2 of Hearts"],
    'Num_of_Hits': [0],
    'Seat_At_Table':[2]
})
pd_serving_dataset

tf_serving_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(pd_serving_dataset)

In [ ]:
pred = reloaded.predict(tf_serving_dataset, verbose=0)[:1][0][0]
print(pred)